# Extract Datasets and Establish Benchmark

**Learning Objectives**
- Divide into Train, Evaluation and Test datasets
- Understand why we need each
- Pull data out of BigQuery and into CSV
- Establish Rules Based Benchmark

## Introduction 
In the previous notebook we demonstrated how to do ML in BigQuery. However BQML is limited to linear models.

For advanced ML we need to pull the data out of BigQuery and load it into a ML Framework, in our case TensorFlow.

While TensorFlow [can read from BigQuery directly](https://www.tensorflow.org/api_docs/python/tf/contrib/cloud/BigQueryReader), the performance is slow. The best practice is to first stage the BigQuery files as .csv files, and then read the .csv files into TensorFlow. 

The .csv files can reside on local disk if we're training locally, but if we're training in the cloud we'll need to move the .csv files to the cloud, in our case Google Cloud Storage.

### Set up environment variables and load necessary libraries

In [1]:
PROJECT = "qwiklabs-gcp-ml-3b4c41182bb8"  # Replace with your PROJECT
REGION = "us-central1"            # Choose an available region for Cloud MLE

In [2]:
import os
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION

In [3]:
!pip freeze | grep google-cloud-bigquery==1.21.0 || pip install google-cloud-bigquery==1.21.0

google-cloud-bigquery==1.21.0


In [4]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


## Review

In the [a_sample_explore_clean](a_sample_explore_clean.ipynb) notebook we came up with the following query to extract a repeatable and clean sample: 
<pre>
#standardSQL
SELECT
  (tolls_amount + fare_amount) AS fare_amount, -- label
  pickup_datetime,
  pickup_longitude, 
  pickup_latitude, 
  dropoff_longitude, 
  dropoff_latitude
FROM
  `nyc-tlc.yellow.trips`
WHERE
  -- Clean Data
  trip_distance > 0
  AND passenger_count > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  -- repeatable 1/5000th sample
  AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))),5000) = 1
  </pre>
  
We will use the same query **with one change**. Instead of using `pickup_datetime` as is, we will extract `dayofweek` and `hourofday` from it. This is to give us some categorical features in our dataset so we can illustrate how to deal with them when we get to feature engineering. The new query will be:

<pre>
SELECT
  (tolls_amount + fare_amount) AS fare_amount, -- label
  EXTRACT(DAYOFWEEK from pickup_datetime) AS dayofweek,
  EXTRACT(HOUR from pickup_datetime) AS hourofday,
  pickup_longitude, 
  pickup_latitude, 
  dropoff_longitude, 
  dropoff_latitude
-- rest same as before
</pre>

## Split into train, evaluation, and test sets

For ML modeling we need not just one, but three datasets.

**Train:** This is what our model learns on

**Evaluation (aka Validation):** We shouldn't evaluate our model on the same data we trained on because then we couldn't know whether it was memorizing the input data or whether it was generalizing. Therefore we evaluate on the evaluation dataset, aka validation dataset.

**Test:** We use our evaluation dataset to tune our hyperparameters (we'll cover hyperparameter tuning in a future lesson). We need to know that our chosen set of hyperparameters will work well for data we haven't seen before because in production, that will be the case. For this reason, we create a third dataset that we never use during the model development process. We only evaluate on this once our model development is finished. Data scientists don't always create a test dataset (aka holdout dataset), but to be thorough you should.

We can divide our existing 1/5000th sample three ways 70%/15%/15%  (or whatever split we like) with some modulo math demonstrated below.

Because we are using a hash function these results are deterministic, we'll get the same exact split every time the query is run (assuming the underlying data hasn't changed)

#### **Exercise 1**

The `create_query` function below returns a query string that we will pass to BigQuery to collect our data. It takes as arguments the phase (`TRAIN`, `VALID`, or `TEST`) and the sample_size (relating to the fraction of the data we wish to sample). Complete the code below so that when the phase is set as `VALID` or `TEST` a new 15% split of the data will be created.

In [6]:
def create_query(phase, sample_size):
    basequery = """
    SELECT
        (tolls_amount + fare_amount) AS fare_amount,
        EXTRACT(DAYOFWEEK from pickup_datetime) AS dayofweek,
        EXTRACT(HOUR from pickup_datetime) AS hourofday,
        pickup_longitude AS pickuplon,
        pickup_latitude AS pickuplat,
        dropoff_longitude AS dropofflon,
        dropoff_latitude AS dropofflat
    FROM
        `nyc-tlc.yellow.trips`
    WHERE
        trip_distance > 0
        AND fare_amount >= 2.5
        AND pickup_longitude > -78
        AND pickup_longitude < -70
        AND dropoff_longitude > -78
        AND dropoff_longitude < -70
        AND pickup_latitude > 37
        AND pickup_latitude < 45
        AND dropoff_latitude > 37
        AND dropoff_latitude < 45
        AND passenger_count > 0
        AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), EVERY_N) = 1
    """

    if phase == "TRAIN":
        subsample = """
        AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), EVERY_N * 100) >= (EVERY_N * 0)
        AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), EVERY_N * 100) <  (EVERY_N * 70)
        """
    elif phase == "VALID":
        subsample = """
        AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), EVERY_N * 100) >= (EVERY_N * 70)
        AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), EVERY_N * 100) <  (EVERY_N * 85)
        """
    elif phase == "TEST":
        subsample = """
        AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), EVERY_N * 100) >= (EVERY_N * 85)
        """

    query = basequery + subsample
    return query.replace("EVERY_N", sample_size)

## Write to CSV
Now let's execute a query for train/valid/test and write the results to disk in csv format. We use Pandas's `.to_csv()` method to do so.

#### **Exercise 2**

The `for` loop below will generate the TRAIN/VALID/TEST sampled subsets of our dataset. Complete the code in the cell below to 1) create the BigQuery `query_string` using the `create_query` function you completed above, taking our original 1/5000th of the dataset and 2) load the BigQuery results of that `query_string` to a DataFrame labeled `df`. 

The remaining lines of code write that DataFrame to a csv file with the appropriate naming.

In [8]:
from google.cloud import bigquery
bq = bigquery.Client(project=PROJECT)

for phase in ["TRAIN", "VALID", "TEST"]:
    # 1. Create query string
    query_string = create_query(phase, '5000')

    # 2. Load results into DataFrame
    df = bq.query(query_string).to_dataframe()

    # 3. Write DataFrame to CSV
    df.to_csv("taxi-{}.csv".format(phase.lower()), index_label = False, index = False)
    print("Wrote {} lines to {}".format(len(df), "taxi-{}.csv".format(phase.lower())))

Wrote 151614 lines to taxi-train.csv
Wrote 31820 lines to taxi-valid.csv
Wrote 34080 lines to taxi-test.csv


In [14]:
len(df[df["hourofday"] == 0])

1489

Note that even with a 1/5000th sample we have a good amount of data for ML. 150K training examples and 30K validation.

<h3> Verify that datasets exist </h3>

In [9]:
!ls -l *.csv

-rw-r--r-- 1 jupyter jupyter 1799474 Nov 20 00:50 taxi-test.csv
-rw-r--r-- 1 jupyter jupyter 7986353 Nov 20 00:49 taxi-train.csv
-rw-r--r-- 1 jupyter jupyter 1673742 Nov 20 00:49 taxi-valid.csv


### Preview one of the files

In [16]:
!head taxi-train.csv

fare_amount,dayofweek,hourofday,pickuplon,pickuplat,dropofflon,dropofflat
31.83,1,0,-73.990922,40.739717,-73.896862,40.758007
14.9,5,0,-74.009155,40.70658,-73.957935,40.717612
8.1,4,0,-73.997545,40.718858,-74.005347,40.74098
3.3,2,0,-74.000522,40.737582,-74.002707,40.734017
11.3,7,0,-73.970422,40.759152,-73.986502,40.72214
5.5,5,0,-73.987683,40.765372,-73.98709,40.755471
10.5,2,0,-73.949372,40.781167,-73.976555,40.745115
27.07,4,0,-73.844595,40.721498,-73.978057,40.746217
22.5,2,0,-73.944457,40.752312,-73.979292,40.76537


Looks good! We now have our ML datasets and are ready to train ML models, validate them and test them.

## Establish rules-based benchmark

Before we start building complex ML models, it is a good idea to come up with a simple rules based model and use that as a benchmark. After all, there's no point using ML if it can't beat the traditional rules based approach!

Our rule is going to be to divide the mean fare_amount by the mean estimated distance to come up with a rate and use that to predict. 

Recall we can't use the actual `trip_distance` because we won't have that available at prediction time (depends on the route taken), however we do know the users pick up and drop off location so we can use euclidean distance between those coordinates.

#### **Exercise 3**

In the code below, we create a rules-based benchmark and measure the Root Mean Squared Error against the label. The function `euclidean_distance` takes as input a Pandas dataframe and should measure the straight line distance between the pickup location and the dropoff location. Complete the code so that the function returns Euclidean distance between the pickup and dropoff location. 

The `compute_rmse` funciton takes the actual (label) value and the predicted value and computes the Root Mean Squared Error between the the two. Complete the code below for the `compute_rmse` function.

In [18]:
import pandas as pd
pd.read_csv("taxi-train.csv")

,fare_amount,dayofweek,hourofday,pickuplon,pickuplat,dropofflon,dropofflat
0,31.83,1,0,-73.990922,40.739717,-73.896862,40.758007
1,14.90,5,0,-74.009155,40.706580,-73.957935,40.717612
2,8.10,4,0,-73.997545,40.718858,-74.005347,40.740980
3,3.30,2,0,-74.000522,40.737582,-74.002707,40.734017
4,11.30,7,0,-73.970422,40.759152,-73.986502,40.722140
...,...,...,...,...,...,...,...
151609,6.00,6,23,-73.990992,40.754265,-73.977942,40.752890
151610,5.50,5,23,-73.981045,40.767757,-73.988707,40.774187
151611,6.50,3,23,-73.993324,40.724505,-74.003273,40.739214
151612,8.50,7,23,-73.971425,40.760290,-73.983005,40.750752


In [20]:
3**2

9

In [26]:
import pandas as pd

def euclidean_distance(df):
    return ((df["pickuplon"] - df["dropofflon"])**2 + (df["pickuplat"] - df["dropofflat"])**2)**(1/2)

def compute_rmse(actual, predicted):
    return ((actual - predicted) ** 2).mean() ** (1/2)

def print_rmse(df, rate, name):
    print("{} RMSE = {}".format(compute_rmse(df["fare_amount"], rate * euclidean_distance(df)), name))

df_train = pd.read_csv("taxi-train.csv")
df_valid = pd.read_csv("taxi-valid.csv")

rate = df_train["fare_amount"].mean() / euclidean_distance(df_train).mean()

print_rmse(df_train, rate, "Train")
print_rmse(df_valid, rate, "Valid") 

7.823827497088752 RMSE = Train
7.704156451026708 RMSE = Valid


The simple distance-based rule gives us an RMSE of <b>$7.70</b> on the validation dataset.  We have to beat this, of course, but you will find that simple rules of thumb like this can be surprisingly difficult to beat. 

You don't want to set a goal on the test dataset because you'll want to tweak your hyperparameters and model architecture to get the best validation error. Then, you can evaluate ONCE on the test data.

## Challenge exercise

Let's say that you want to predict whether a Stackoverflow question will be acceptably answered. Using this [public dataset of questions](https://console.cloud.google.com/marketplace/details/stack-exchange/stack-overflow?filter=solution-type:dataset&q=stack%20overflow), create a machine learning dataset that you can use for classification.
<p>
What is a reasonable benchmark for this problem?
What features might be useful?
<p>
If you got the above easily, try this harder problem: you want to predict whether a question will be acceptably answered within 2 days. How would you create the dataset?
<p>
Hint (highlight to see):
<p style='color:white' linkstyle='color:white'> 
You will need to do a SQL join with the table of [answers]( https://bigquery.cloud.google.com/table/bigquery-public-data:stackoverflow.posts_answers) to determine whether the answer was within 2 days.
</p>

Copyright 2019 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.